### Imports

In [1]:
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

# Reading file
eaz = gpd.read_file('atx_eaz_multi_year.geojson')

# Convert to GeoJSON for Plotly
geojson = eaz.__geo_interface__

### Data manipulation

In [2]:
# Variables to be visualized
variables = ['indexed_vulnerability', 'pct_poc', 'pct_underserved_poc', 'median_hh_inc', 'pct_with_disability', 'pct_food_stamps', 'pct_rent_over35', 'pct_wo_broadband', 'pct_less_vehicles', 'pct_over_70']

In [3]:
# Reshaping the dataframe
eaz_wide = eaz.pivot_table(index='GEOID', columns='acs_year', values=variables)

# Renaming the columns, since currently multiindex
eaz_wide.columns = [f'{var}_{year}' for var, year in eaz_wide.columns]

# Reset index to make 'GEOID' a column again
eaz_wide.reset_index(inplace=True)

In [4]:
# Identifying first and last year
min_year = eaz['acs_year'].min()
max_year = eaz['acs_year'].max()

In [5]:
# Creating difference since 2019 columns
for var in variables:
    for year in range(min_year, max_year + 1):
        base_var = f'{var}_{min_year}'
        compare_var = f'{var}_{year}'
        diff_var = f'difference_since_2019_{year}_{var}'
        eaz_wide[diff_var] = eaz_wide[compare_var] - eaz_wide[base_var]
    

In [6]:
# Keeping columns that start with 'GEOID' or 'difference_since'
columns_to_keep = eaz_wide.filter(regex='^(GEOID|difference_since)').columns
eaz_wide_filtered = eaz_wide[columns_to_keep]

In [7]:
# Reshaping to long format
df_long = eaz_wide_filtered.melt(id_vars=['GEOID'], var_name='variable', value_name='value')

In [8]:
# Extract the correct year and base variable
df_long[['ignore', 'acs_year', 'base_var']] = df_long['variable'].str.extract(r'(difference_since_2019)_(\d{4})_(.*)')

# Recreating variable name with the proper prefix
df_long['base_var'] = 'difference_since_2019_' + df_long['base_var']

In [9]:
# Filtering out the rows we are interested in
df_filtered = df_long[df_long['base_var'].isin(['difference_since_2019_' + var for var in variables])]

In [10]:
# Pivot the data to have separate columns for each variable
df_pivot = df_filtered.pivot_table(
    index=['GEOID', 'acs_year'], 
    columns='base_var', 
    values='value'
).reset_index()

In [11]:
# Sorting the dataframe
df_sorted = df_pivot.sort_values(by=['acs_year', 'GEOID'], ascending=[True, True])

In [12]:
# Merging the two dataframes together
eaz_filtered = eaz.drop(columns=['difference_since_2019'], inplace=False)

In [13]:
df_sorted["acs_year"] = pd.to_numeric(df_sorted["acs_year"])

# Merging with original dataframe to restore geometry information
merged_data= eaz_filtered.merge(df_sorted, on=["GEOID","acs_year"])

At this point, I realized that the ideal data format would be long. 
A variable column for the `variables`
Year
Dimension - absolute vs relative (difference since 2019)

In [14]:
# Reshaping data to be in long format
absolute_columns = variables
relative_columns = ['difference_since_2019_' + var for var in variables]

In [15]:
# Reshaping
absolute_melted = merged_data.melt(id_vars=['GEOID', 'acs_year', 'geometry'], 
                                   value_vars=absolute_columns,
                                   var_name='variable',
                                   value_name='value')

# Adding dimension column
absolute_melted['dimension'] = 'absolute'

relative_melted = merged_data.melt(id_vars=['GEOID', 'acs_year', 'geometry'],
                                   value_vars=relative_columns,
                                   var_name='variable',
                                   value_name='value')

relative_melted['dimension'] = 'relative'

# Removing the difference_since_2019 prefix from the data
relative_melted['variable'] = relative_melted['variable'].str.replace('difference_since_2019_', '')

# Concatenating the data together
long_data = pd.concat([absolute_melted, relative_melted])

# Sorting the dataframe
long_data = long_data.sort_values(by=['GEOID', 'acs_year', 'variable', 'dimension'], ascending=[True, True, True, True])

In [16]:
long_data

,GEOID,acs_year,geometry,variable,value,dimension
186,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",indexed_vulnerability,57.295374,absolute
186,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",indexed_vulnerability,0.000000,relative
6222,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",median_hh_inc,-76424.000000,absolute
6222,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",median_hh_inc,0.000000,relative
10246,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",pct_food_stamps,0.065128,absolute
...,...,...,...,...,...,...
6035,48491021603,2022,"POLYGON ((-97.72998 30.75761, -97.72950 30.758...",pct_underserved_poc,0.083445,relative
10059,48491021603,2022,"POLYGON ((-97.72998 30.75761, -97.72950 30.758...",pct_with_disability,0.999011,absolute
10059,48491021603,2022,"POLYGON ((-97.72998 30.75761, -97.72950 30.758...",pct_with_disability,0.001812,relative
16095,48491021603,2022,"POLYGON ((-97.72998 30.75761, -97.72950 30.758...",pct_wo_broadband,0.097264,absolute


In [17]:
# Merging to get eaz_type


# This code is probably not required
eaz_eaz_type = eaz[['GEOID', 'eaz_type']]

long_data = pd.merge(long_data, eaz_eaz_type, on='GEOID', how='left')

In [18]:
long_data

,GEOID,acs_year,geometry,variable,value,dimension,eaz_type
0,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",indexed_vulnerability,57.295374,absolute,Medium Vulnerable
1,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",indexed_vulnerability,57.295374,absolute,Medium Vulnerable
2,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",indexed_vulnerability,57.295374,absolute,Medium Vulnerable
3,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",indexed_vulnerability,57.295374,absolute,Medium Vulnerable
4,48021950101,2019,"POLYGON ((-97.49111 30.21237, -97.49098 30.212...",indexed_vulnerability,0.000000,relative,Medium Vulnerable
...,...,...,...,...,...,...,...
160955,48491021603,2022,"POLYGON ((-97.72998 30.75761, -97.72950 30.758...",pct_wo_broadband,0.097264,absolute,Medium-Low Vulnerable
160956,48491021603,2022,"POLYGON ((-97.72998 30.75761, -97.72950 30.758...",pct_wo_broadband,-0.017887,relative,Medium-Low Vulnerable
160957,48491021603,2022,"POLYGON ((-97.72998 30.75761, -97.72950 30.758...",pct_wo_broadband,-0.017887,relative,Medium-Low Vulnerable
160958,48491021603,2022,"POLYGON ((-97.72998 30.75761, -97.72950 30.758...",pct_wo_broadband,-0.017887,relative,Medium-Low Vulnerable


In [19]:
# Data for the bar plot
eaz_type = eaz[['GEOID', 'eaz_type', 'acs_year']]

# Dataframe containing NAME
name = eaz[['GEOID', 'NAME']]

# Merging name information
long_data = pd.merge(long_data, name, on='GEOID', how='left')

### Creating plotly visualization

In [20]:
# Labels for dropdown options
variable_labels = {
    'indexed_vulnerability': 'Indexed Vulnerability',
    'pct_poc': 'Percentage People of Color',
    'pct_underserved_poc': 'Percentage Underserved People of Color',
    'median_hh_inc': 'Median Household Income',
    'pct_with_disability': 'Percentage with Disability',
    'pct_food_stamps': 'Percentage on Food Stamps',
    'pct_rent_over35': 'Percentage Rent over 35%',
    'pct_wo_broadband': 'Percentage without Broadband',
    'pct_less_vehicles': 'Percentage with Less Vehicles',
    'pct_over_70': 'Percentage over 70'
}

dimension_labels = {
    'absolute': 'Absolute',
    'relative': 'Difference since 2019'
}

# Ordering for bar plot
order = ['Least Vulnerable', 'Medium-Low Vulnerable', 'Medium Vulnerable', 'Medium-High Vulnerable', 'Most Vulnerable']
order_mapping = {name: i for i, name in enumerate(order)}

In [23]:
# Create the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout of the Dash app
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Equity Analysis Zones", className="mb-3"), width=12)
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Label("Select Variable"),
            dcc.Dropdown(
                id='variable-dropdown',
                options=[{'label': variable_labels[var], 'value': var} for var in long_data['variable'].unique()],
                value='indexed_vulnerability',
                className="mb-3",
                clearable=False
            )
        ], width=4),
        dbc.Col([
            dbc.Label("Select Dimension"),
            dcc.Dropdown(
                id='dimension-dropdown',
                options=[{'label': dimension_labels[dim], 'value': dim} for dim in long_data['dimension'].unique()],
                value='absolute',
                className="mb-3",
                clearable=False
            )
        ], width=4),
        dbc.Col([
            dbc.Label("Select Year"),
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': str(year), 'value': year} for year in long_data['acs_year'].unique()],
                value=2019,
                className="mb-3",
                clearable=False
            )
        ], width=4)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='choropleth-map'), width=6),
        dbc.Col(dcc.Graph(id='bar-plot'), width=6)        
    ])
], fluid=True)

# Define the callback to update the plot based on dropdown selections
@app.callback(
    [Output('choropleth-map', 'figure'),
     Output('bar-plot', 'figure')],
    [Input('variable-dropdown', 'value'),
     Input('dimension-dropdown', 'value'),
     Input('year-dropdown', 'value')]
)
def update_plots(selected_variable, selected_dimension, selected_year):
    filtered_data = long_data[(long_data['variable'] == selected_variable) & 
                       (long_data['dimension'] == selected_dimension) & 
                       (long_data['acs_year'] == selected_year)]
    
    

    title = f"{selected_variable}-{selected_dimension}-{selected_year}"

    # Choropleth map
    choropleth_fig = go.Figure()
    choropleth_fig.add_trace(go.Choroplethmapbox(
        locations=filtered_data['GEOID'],
        geojson=geojson,
        featureidkey="properties.GEOID",
        z=filtered_data['value'],
        colorscale="Reds",
        zmin=filtered_data['value'].min(),
        zmax=filtered_data['value'].max(),
        marker_opacity=0.8,
        marker_line_width=0.25,
        colorbar=dict(
            x = 0.1,
            y = -0.015,
            thickness=10,
            outlinecolor='rgba(0,0,0,0)',  # Transparent outline
            bgcolor='rgba(255,255,255,1)', # White background
            len = 0.2,
            orientation = 'h'
        )
    ))

    choropleth_fig.update_layout(
        title_text="Equity Analysis Zones<br><sup>Indexed Vulnerability Score</sup>",
        title_font=dict(size=25),
        title_x = 0.5,
        mapbox_style="carto-positron",
        # width=1200,
        # height=800,
        mapbox_zoom=8,
        mapbox_center={"lat": 30.2672, "lon": -97.7431}
    )

    # Bar plot
    filtered_data_bar = eaz_type[(eaz_type['acs_year'] == selected_year)]
    bar_data = filtered_data_bar[['eaz_type']].value_counts().reset_index()
    bar_data.columns = ['eaz_type', 'count']
    bar_data['order'] = bar_data['eaz_type'].map(order_mapping)
    bar_data = bar_data.sort_values('order')  # Sort by the specified order
    bar_fig = go.Figure()
    bar_fig.add_trace(go.Bar(
        x=bar_data['eaz_type'],
        y=bar_data['count'],
        name=title
    ))
    bar_fig.update_layout(
        title=f'Distribution of census tracts-{selected_year}'
    ) #, xaxis_title='ID', yaxis_title='Value')

    bar_fig.update_xaxes(
        tickmode='array',
        tickvals=[0, 1, 2, 3, 4],
        ticktext=['Least<br>Vulnerable', 'Medium-Low<br>Vulnerable', 'Medium<br>Vulnerable', 'Medium-High<br>Vulnerable', 'Most<br>Vulnerable'],
        tickfont=dict(size=11),
        tickangle=0
)

    return choropleth_fig, bar_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)